In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory



llm = ChatOpenAI(
    model_name="gpt-4o-mini-2024-07-18",
    temperature=0.2,
)

memory = ConversationBufferMemory(
    return_messages=True,
    input_key="question",
    memory_key="chat_history"
)


cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = TextLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()


map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim. If there is no relevant text, return : ''
            ---
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm


def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    return "\n\n".join(
        map_doc_chain.invoke(
            {"context": doc.page_content, "question": question}
        ).content
        for doc in documents
    )


map_chain = {
    "documents": retriever,
    "question": RunnablePassthrough(), 
} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            
            Chat History:
            {chat_history}
            """,
        ),
        ("human", "{question}"),
    ]
)
final_chain = LLMChain(
    llm=llm,
    prompt=final_prompt,
    memory=memory,
)
chain = {"context": map_chain, "question": RunnablePassthrough()} | final_chain

questions = [
    "Is Aaronson guilty?",
    "What message did he write in the table?",
    "Who is Julia?"
]

for question in questions:
    print(f"질문: {question}")
    response = chain.invoke(question)
    print(f"답변: {response['text']}")
    print("-" * 3)


/var/folders/bb/hwq64p2n5rl8w8dygq6q1xjh0000gn/T/ipykernel_32927/4267276332.py:14: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(
/var/folders/bb/hwq64p2n5rl8w8dygq6q1xjh0000gn/T/ipykernel_32927/4267276332.py:19: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(
/var/folders/bb/hwq64p2n5rl8w8dygq6q1xjh0000gn/T/ipykernel_32927/4267276332.py:37: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used ins

질문: Is Aaronson guilty?
답변: Yes, Aaronson was guilty of the crimes he was charged with.
---
질문: What message did he write in the table?
답변: He wrote: "FREEDOM IS SLAVERY," "TWO AND TWO MAKE FIVE," and "GOD IS POWER."
---
질문: Who is Julia?
답변: Julia is a character significant to Winston, the protagonist of the text. She represents a personal relationship that Winston values deeply, despite the oppressive regime of the Party. Julia is portrayed as someone Winston loves, and their bond is emphasized through his emotional responses and memories of her.
---
